# Image Quality Transfer (IQT) in MRI with Deep Neural Networks

- MediCSS Project
- 03/07/2019
- v1.1
- Led by Harry Hongxiang Lin and Ryutaro Tanno
- Language: Python+Keras+Tensorflow

This notebook is a demo for IQT using deep learning. IQT predicts cross-plane voxels along slice thickness direction by learning a super-resolution deep neural network from low/high-resolution pair data. These pre-trained networks were trained by the Human Connetome Project (HCP) dataset and are already available in the project folder.

The project aims to do super-resolution study on MR images. You will gain more practical knowledge of deep learning, especially on:
- Understanding a general structure of pipeline for a deep learning task: data preprocessing-training-test-evaluation;
- Apply external datasets to the pre-trained networks;
- Visualise and evaluate the neural network in order to have some intuitive for deep learning.

## Prerequisites

### Material
1. [Provided] Human Connectome Project dataset: http://www.humanconnectomeproject.org/ .
2. Pre-trained models: 3D SR U-net `srunet16_16_2_nf4` and Anisotropic U-net `anisounet16_16_2_nf4`.

### Config `config.py`

Configurate `config.py` in the IQT project folder:
1. In `general_configuration`, substitute `<YOUR HOME DIRECTORY>` in `dataset_path` and `base_path` with your home directory path. For example, here is my configuration:
    `'dataset_path' : '/home/harrylin/'` and `'base_path' : '/home/harrylin/tutorial_result'`.
2. Put the pretrained models, `srunet16_16_2_nf4` and `anisounet16_16_2_nf4`, into the `base_path`. Then set `'job_name' : 'srunet16_16_2_nf4'` or `'job_name' : 'anisounet16_16_2_nf4'`.
3. [Done] Set `dataset_info` for the new including dataset. An example for `HCP-Wu-Minn-Contrast` dataset is given.
4. In `training_configuration`, choose `approach` and `dataset` for training/test. The alternatives for `approach`: `SRUnet` and `AnisoUnet` corresponding to the pretrained models `'srunet16_16_2_nf4'` and `'anisounet16_16_2_nf4'`, respectively.

## Data preparation
- The module loads the technical configuration on dataset and neural networks.
- Create folders for placing results.


In [1]:
from config import general_configuration as gen_conf
from config import training_configuration as train_conf
from workflow.data_preparation import data_preparation

# pass arguments to jupyter notebook
import sys
sys.argv = ['main.py', '--gpu', '0']

# data preparation
opt, gen_conf, train_conf = data_preparation(gen_conf, train_conf)

## Input data simulation
- Downsample HCP dataset
- The simulated downsampling data will be saved in the folder of each subject `<dataset_path>/<path>/subject...`.

In [2]:
from utils.preprocessing_util import preproc_dataset
preproc_dataset(gen_conf, train_conf)

Downsample HCP data with 1D Gaussian filter.
Processing '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/202719/T1w/T1w_acpc_dc_restore_brain_sim036T_ds6_gap2_groundtruth.nii'
Save to '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/202719/T1w/T1w_acpc_dc_restore_brain_procin.nii'
Processing '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/204218/T1w/T1w_acpc_dc_restore_brain_sim036T_ds6_gap2_groundtruth.nii'
Save to '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/204218/T1w/T1w_acpc_dc_restore_brain_procin.nii'
Processing '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/200210/T1w/T1w_acpc_dc_restore_brain_sim036T_ds6_gap2_groundtruth.nii'
Save to '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/200210/T1w/T1w_acpc_dc_restore_brain_procin.nii'
Processing '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/202113/T1w/T1w_acpc_dc_restore_brain_sim036T_ds6_gap2_groundtruth.nii'
Save to '/Users/hongxianglin/document/ucl_coding/P

True

### Example: Visualize the downsampled MRI images

Please select one subject from the training set and visualize one downsampled image (input) together with its original image (output) on axial/coronal/sagittal views. Suggest using Python libs: `nibabel` (https://nipy.org/nibabel/coordinate_systems.html) `matplotlib` (https://matplotlib.org/gallery/index.html) to plot.

In [5]:
## Sample answer.

# load the subject 200008
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline

# image file paths
## gt=ground truth, proc = pre-processed
gt_img_path = '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/200008/T1w/T1w_acpc_dc_restore_brain.nii'
proc_img_path = '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/200008/T1w/T1w_acpc_dc_restore_brain_procin.nii'

gt_img = nib.load(gt_img_path).get_data()
proc_img = nib.load(proc_img_path).get_data()

# Plot images in a 2x3 grid. Cross-sectional views by row w/o downsampling by column.
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10*2, 10*2),
                        subplot_kw={'xticks': [], 'yticks': []})
fig.subplots_adjust(left=0.03, right=0.97, hspace=0.05, wspace=0.05)

## Intersection of 3 viewing planes (axial/coronal/sagittal) 
# intersection coordinate at original image
gt_intersect = [150, 150, 128]
# sparse scale
sparse_scale = gen_conf['dataset_info']['HCP-Wu-Minn-Contrast']['sparse_scale']
apsect_ratio_scale = np.prod(sparse_scale)/np.array(sparse_scale)
# intersection coordinate at processed image
proc_intersect = np.array(gt_intersect)/np.array(sparse_scale)

for idx, ax in enumerate(axs):
    # initialisation
    gt_coord = [slice(None), slice(None), slice(None)]
    proc_coord = [slice(None), slice(None), slice(None)]
    
    # 
    gt_coord[idx] = slice(gt_intersect[idx], gt_intersect[idx]+1, 1)
    proc_coord[idx] = slice(proc_intersect[idx], proc_intersect[idx]+1, 1)
    
    gt_reduced_img = np.squeeze(org_img[org_coord])
    ax[0].imshow(org_reduced_img.T, interpolation=None, cmap='gray', aspect=1, origin='lower')
    
    proc_reduced_img = np.squeeze(proc_img[proc_coord])
    ax[1].imshow(proc_reduced_img.T, interpolation=None, cmap='gray', aspect=apsect_ratio_scale[idx], origin='lower')


FileNotFoundError: No such file: '/Users/hongxianglin/document/ucl_coding/P5_mri_image/HCP/200008/T1w/T1w_acpc_dc_restore_brain.nii'

## Test stage

We provide a SRU-net pre-trained from the Human Connectome Project (HCP) dataset. 

In [3]:
from workflow.test import testing

model = testing(gen_conf, train_conf)

Using TensorFlow backend.


Start testing ... load data, mean, std and model...
/Users/hongxianglin/document/ucl_coding/P3_IQT_Unet/tutorial_result/srunet16_16_2_nf4/models/HCP-Wu-Minn-Contrast/1-SRUnet-3-(32, 32, 4)-(16, 16, 2)_mean.npz
/Users/hongxianglin/document/ucl_coding/P3_IQT_Unet/tutorial_result/srunet16_16_2_nf4/models/HCP-Wu-Minn-Contrast/1-SRUnet-3-(32, 32, 4)-(16, 16, 2)_std.npz
/Users/hongxianglin/document/ucl_coding/P3_IQT_Unet/tutorial_result/srunet16_16_2_nf4/models/HCP-Wu-Minn-Contrast/1-SRUnet-3-(32, 32, 4)-(16, 16, 2).h5
Test model ...
(5780, 1, 32, 32, 4)
5780/5780 [==============================] - 5675s 982ms/step
(5780, 1, 32, 32, 32)
Reconstructing ...
/Users/hongxianglin/document/ucl_coding/P3_IQT_Unet/tutorial_result/srunet16_16_2_nf4/result/HCP-Wu-Minn-Contrast/202113_T1w-SRUnet-3-(32, 32, 4)-(16, 16, 2).nii
Save file at /Users/hongxianglin/document/ucl_coding/P3_IQT_Unet/tutorial_result/srunet16_16_2_nf4/result/HCP-Wu-Minn-Contrast/202113_T1w-SRUnet-3-(32, 32, 4)-(16, 16, 2).nii
(5780

KeyboardInterrupt: 

### Assignments:

1. Show summary of networks, which is saved in the variable `model` (Hint: Using `model.summary()` https://keras.io/models/about-keras-models/).
2. Visualise the activation for each layer (Hint: https://keras.io/examples/conv_filter_visualization/).
3. Use other datasets (e.g. HBN, IBSR) for predicting the network outputs.

HBN: Child Mind Institute Healthy Brain Network http://fcon_1000.projects.nitrc.org/indi/cmi_healthy_brain_network/index.html

IBSR: Internet Brain Segmentation Repository https://www.nitrc.org/projects/ibsr/

In [ ]:
# Write the answer here.

## Evaluation
Evaluation metrics:
1. PSNR: Peak Signal-to-Noise Ratio
2. SSIM: Structural Similarity Index
3. RMSE: Root Mean Squared Error

### Assigments:
1. Look at the evaluation result under `</task_name/>/evaluation/stats_brain.csv` and compare results from the different networks.
2. Read the reference on SSIM, https://www.cns.nyu.edu/pub/lcv/wang03-preprint.pdf, and intuitively and/or theoretically explain if this index is suitable to MR image quality assessment.


In [4]:
from workflow.evaluation import evaluation

evaluation(gen_conf, train_conf)

/Users/hongxianglin/document/ucl_coding/P3_IQT_Unet/tutorial_result/srunet16_16_2_nf4/result/HCP-Wu-Minn-Contrast/202113_T1w-SRUnet-3-(32, 32, 4)-(16, 16, 2).nii
/Users/hongxianglin/document/ucl_coding/P3_IQT_Unet/tutorial_result/srunet16_16_2_nf4/result/HCP-Wu-Minn-Contrast/201414_T1w-SRUnet-3-(32, 32, 4)-(16, 16, 2).nii


FileNotFoundError: No such file: '/Users/hongxianglin/document/ucl_coding/P3_IQT_Unet/tutorial_result/srunet16_16_2_nf4/result/HCP-Wu-Minn-Contrast/201414_T1w-SRUnet-3-(32, 32, 4)-(16, 16, 2).nii'